In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
# Definiujemy funkcje aktywacji fi (sigmoida)
# fi' = fi*(1-fi) 

def sigmoid(x): return (1+np.exp(-x))**(-1) 

def deriv_sigmoid(x): return sigmoid(x)*(1-sigmoid(x))

# Odwrotna funkcja sigmoidalna - Czy można jej tutaj użyć?
def inv_sigmoid(x): return np.log(x/(1-x))

In [3]:
# Generujemy wagi
def wygeneruj_wagi(wejscie, wyjscie):
    # wyjściem jest liczba neuronów w następnej warstwie
    wektor_wag = np.random.normal(0,1/math.sqrt(len(wejscie)),(1+len(wejscie))*wyjscie)
    return np.reshape(wektor_wag, (wyjscie, 1+len(wejscie) )  )

In [4]:
wygeneruj_wagi((1,2), 2)

array([[-0.35723577,  1.21747645, -1.22620051],
       [-1.25128079,  0.12543397,  0.34059919]])

In [5]:
# Wejscie = wektor 784
# Wyjscie = wektor 10

# nerunony_na_warstwe: podajemy liczbę na warstwy głębokie
def forward_prop(wejscie_dane, liczb_neuronow_wyjscie ):
    """Podajemy dane na wejście, dodajemy na bias = 1 na początek wektora (otrzymujemy w ten sposób X).
    Zadajemy najpierw ile chcemy mieć neuronów na wyjściu
    Nastepnie liczymy net = W*X <- wyjście netto.
    Póżniej nakładamy funkcję aktywacji fi na net, otrzymując fi(net) = a <- wyjście z warstwy.
    Musimy też zapisywać stany x, net, a dla każdej warstwy (np. lista krotek)"""

    stany_x_net_a = []
    # Musimy przerobić X na wektor kolumnowy
    X = np.vstack([1,wejscie_dane.reshape(-1,1)])
    W = wygeneruj_wagi(wejscie_dane, liczb_neuronow_wyjscie)
    net = W @ X
    a = sigmoid(net)
    stany_x_net_a.append((X,net,a))
    return stany_x_net_a


In [6]:
A1 = forward_prop(np.array([1,1,0]), liczb_neuronow_wyjscie=2)[0][2]
print(A1)

A2 = forward_prop(A1, liczb_neuronow_wyjscie=2)[0][2]
print(A2)

[[0.17980467]
 [0.30104507]]
[[0.75236525]
 [0.45737023]]


In [8]:
# Definiujemy propagację wstecz

# wyjscie oczekiwane - to co chcielibyśmy uzyskać
# wyjscie dane - nasz wynik z forward_prop

# Funkcja straty to L = 1/2(a[L]-y)^2, czyli pochodna z L to a[L]-y
def back_prop(wyjscie_oczekiwane, wyjscie_dane, wagi_warstwy, stala_uczenia = 0.01 ):

    # Ostatnia warstwa
   # a_ost = 
    dL_a_wyjscia = wyjscie_dane - wyjscie_oczekiwane

    # mamy pochodną dL/da * fi(net)
    delta = dL_a_wyjscia * deriv_sigmoid( inv_sigmoid(wyjscie_dane) )
    dL_dW = delta * wyjscie_dane.T

    # Liczymy dL/dX i usuwamy pierwszy element, otrzymując dL/da niższej warstwy
    dL_dX = wagi_warstwy.T @ delta

    dL_da_nizsze = dL_dX[1:]

    # Aktualizujemy wagi
    W_nowe = wagi_warstwy - stala_uczenia*dL_dW


In [9]:
A = np.array([[0.1,-0.2,0.3], [-0.4,0.5,-0.6]])
B = np.array([1,1,0])
C= B.reshape(-1,1)
C

array([[1],
       [1],
       [0]])

In [10]:
A@B

array([-0.1,  0.1])

In [11]:
np.hstack([1,A])

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [ ]:
sigmoid(C)

TypeError: only length-1 arrays can be converted to Python scalars